In [2]:
import json

In [3]:
data_path = 'data/doc2dial/doc2dial_dial_train.json'
with open(data_path, 'r') as f:
    data = json.load(f)

In [4]:
data['dial_data'].keys()

dict_keys(['dmv', 'ssa', 'va', 'studentaid'])

In [2]:
data_path = 'doc2dial_doc.json'
with open(data_path, 'r') as f:
    data = json.load(f)

In [3]:
len(data['doc_data']['ssa'].keys())

109

In [4]:
doc_list = list(data['doc_data']['ssa'].keys())

In [5]:
doc_list[0]

'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0'

In [6]:
data['doc_data']['ssa'][doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])